# 環境の説明
```
conda create -n LLMfromScratch python
```
ここからpipを使ってインストールする

インストールしたもの

`pip install tiktoken`

`pip install torch`

# テキストデータの読み込み

urllibを使ってtxtデータに変換する

In [1]:
!pwd

/Users/kakuayato/Documents/GitHub/CurioSync_LLM/public/llm


In [2]:
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt")
file_path = "./data/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)
print(f"File downloaded to {file_path}")

File downloaded to ./data/the-verdict.txt


読み込めた

In [3]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()
print(f"Text length: {len(raw_text)}")
print(f"Text snippet: {raw_text[:100]}")  # Print the first 100 characters of the text

Text length: 20479
Text snippet: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [4]:
import re 
text = "Hello, world. this, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'this,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result_2 = re.split(r'([,.]|\s)',text)
print(result_2)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'this', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
result_2_2 = [item for item in result_2 if item.strip()]
print(result_2_2)

['Hello', ',', 'world', '.', 'this', ',', 'is', 'a', 'test', '.']


In [7]:
new_text = "Hello, world. Is this-- a test?"
result_3 = re.split(r'([,.:;?_!"()\']|--|\s)', new_text)
result_3 = [item for item in result_3 if item.strip()]
print(result_3)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(f"Preprocessed text length: {len(preprocessed)}")
print(preprocessed[:40])  # Print the first 10 tokens

Preprocessed text length: 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his']


### トークンをトークンIDに変換する

In [9]:
all_words = sorted(set(preprocessed))
print(f"Number of unique tokens: {len(all_words)}")

Number of unique tokens: 1130


In [10]:
vocab = {token:integer for integer,token in enumerate(all_words)}
'''

'''
print(f"Vocabulary size: {len(vocab)}")
print("vocab", vocab)

Vocabulary size: 1130
vocab {'!': 0, '"': 1, "'": 2, '(': 3, ')': 4, ',': 5, '--': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'Ah': 12, 'Among': 13, 'And': 14, 'Are': 15, 'Arrt': 16, 'As': 17, 'At': 18, 'Be': 19, 'Begin': 20, 'Burlington': 21, 'But': 22, 'By': 23, 'Carlo': 24, 'Chicago': 25, 'Claude': 26, 'Come': 27, 'Croft': 28, 'Destroyed': 29, 'Devonshire': 30, 'Don': 31, 'Dubarry': 32, 'Emperors': 33, 'Florence': 34, 'For': 35, 'Gallery': 36, 'Gideon': 37, 'Gisburn': 38, 'Gisburns': 39, 'Grafton': 40, 'Greek': 41, 'Grindle': 42, 'Grindles': 43, 'HAD': 44, 'Had': 45, 'Hang': 46, 'Has': 47, 'He': 48, 'Her': 49, 'Hermia': 50, 'His': 51, 'How': 52, 'I': 53, 'If': 54, 'In': 55, 'It': 56, 'Jack': 57, 'Jove': 58, 'Just': 59, 'Lord': 60, 'Made': 61, 'Miss': 62, 'Money': 63, 'Monte': 64, 'Moon-dancers': 65, 'Mr': 66, 'Mrs': 67, 'My': 68, 'Never': 69, 'No': 70, 'Now': 71, 'Nutley': 72, 'Of': 73, 'Oh': 74, 'On': 75, 'Once': 76, 'Only': 77, 'Or': 78, 'Perhaps': 79, 'Poor': 80, 'Professional'

### テキストトークナイザの実装
トークン化と逆トークン化（トークンIDからトークンに変換）を実装する

In [11]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # token化された語彙が入ってくる
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        #　指定された句読点の前にあるスペースを削除
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
text_from_ids = tokenizer.decode(ids)
print("ids:", ids)
print("text_from_ids:", text_from_ids)

ids: [56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]
text_from_ids: It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


特別なトークンに対する新しいトークナイザの作成

In [13]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))
print(list(vocab.items())[-5:])

1132
[('younger', 1127), ('your', 1128), ('yourself', 1129), ('<|endoftext|>', 1130), ('<|unk|>', 1131)]


In [14]:
class SimpleTokenizerV2:
    '''
    未知のトークンに対しては全てunkを押しつけるとなっていたので，encodeの仕事だと思う
    '''
    def __init__(self, vocab):
        # token化された語彙が入ってくる
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # if文のところを書き方省略してるっぽい
        preprocessed = [
            item 
            if item in self.str_to_int
            else "<|unk|>" 
            for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        #　指定された句読点の前にあるスペースを削除
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [15]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlist terraces of someunknowPlace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlist terraces of someunknowPlace.


In [16]:
tokenizer2 = SimpleTokenizerV2(vocab)
print(text)
print(tokenizer2.encode(text))
print(tokenizer2.decode(tokenizer2.encode(text)))

Hello, do you like tea? <|endoftext|> In the sunlist terraces of someunknowPlace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 1131, 984, 722, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the <|unk|> terraces of <|unk|>.


In [17]:
word = "you"
if word in vocab:
    print(f"{word}のIDは{vocab[word]}です")

youのIDは1126です


# バイトペアエンコーディング

In [ ]:
# !pip install tiktoken

In [19]:
from importlib.metadata import version 
import tiktoken

print("tiktoken.version:", version("tiktoken"))

tiktoken.version: 0.9.0


In [20]:
tokenizer = tiktoken.get_encoding("gpt2")

In [21]:
text_for_tiktoken = text
print(text_for_tiktoken)

Hello, do you like tea? <|endoftext|> In the sunlist terraces of someunknowPlace.


In [22]:
integers = tokenizer.encode(text_for_tiktoken, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 4868, 8812, 2114, 286, 617, 2954, 2197, 27271, 13]


In [23]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlist terraces of someunknowPlace.


#### 練習問題2-1

In [24]:
text = "Akwirw ier"
integers = tokenizer.encode(text)
print("integers:",integers)
for integer in integers:
    # print(integer)
    print(f"{integer}は{tokenizer.decode([integer])}です．")

strings = tokenizer.decode(integers)
print("strings:", strings)

integers: [33901, 86, 343, 86, 220, 959]
33901はAkです．
86はwです．
343はirです．
86はwです．
220は です．
959はierです．
strings: Akwirw ier


スライディングウィンドウによるデータサンプリング

In [25]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [26]:
enc_sample = enc_text[50:]

In [27]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:       {y}")

x: [290, 4920, 2241, 287]
y:       [4920, 2241, 287, 257]


In [28]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [30]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


データローダーの作成

In [ ]:
# !pip install torch

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 97.6 MB/s eta 0:00:00ta 0:00:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [40]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        


In [45]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length,stride)

    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [50]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)
data_iter= iter(dataloader)
first_batch = next(data_iter)
second_batch = next(data_iter)
print(first_batch)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [55]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=5,
    max_length=4,
    stride=4,
    shuffle=False
)
data_iter= iter(dataloader)
first_batch = next(data_iter)
second_batch = next(data_iter)
print("first_batch:",first_batch)
print("second_batch:",second_batch)

first_batch: [tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568]])]
second_batch: [tensor([[  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11],
        [  287,   262,  6001,   286],
        [  465, 13476,    11,   339]]), tensor([[  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287],
        [  262,  6001,   286,   465],
        [13476,    11,   339,   550]])]


トークン埋め込みを作成する


In [56]:
input_ids = torch.tensor([2,3,5,1])

vocab_size = 6
output_dim = 3

In [60]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)
print(embedding_layer(torch.tensor([3])))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [61]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


埋め込みベクトルに位置情報を追加する

In [63]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(token_embedding_layer)

Embedding(50257, 256)


In [73]:
max_length = 4

dataloader = create_dataloader_v1(
    raw_text,
    batch_size=8,
    max_length=max_length,
    stride = max_length,
    shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


In [81]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
# print(token_embeddings)

torch.Size([8, 4, 256])


In [82]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
# print(pos_embeddings)

torch.Size([4, 256])


In [83]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
# print(input_embeddings)

torch.Size([8, 4, 256])
